In [1]:
%run /home/dbaciur/NTU/NTU/notes/Constants.ipynb
%run /home/dbaciur/NTU/NTU/notes/CommonUtils.ipynb

Constants loaded


In [2]:
import torch
import torchvision as tv

import os
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torchvision
import intervals as I
import moviepy
from pathlib import Path
from moviepy.editor import VideoFileClip

### Load frames

In [4]:
frames_df = pd.read_csv(C.Frames.FRAMES_DF_DRAFT_PATH, converters=C.F_CONVERTERS)
frames_df.head()

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layouts,Layout
0,PH1011-PHYSICS_20150922__3435.269__40__B_CHART...,PH1011-PHYSICS_20150922,3435.269,40,B_CHARTS_P,RA13,"[39, 126]","[B_CHARTS_S, B_CHARTS_P]",3.0,3
1,PH1011-PHYSICS_20150922__3703.298__43__B_CHART...,PH1011-PHYSICS_20150922,3703.298,43,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]",3.0,3
2,PH1011-PHYSICS_20150922__4040.803__44__B_CHART...,PH1011-PHYSICS_20150922,4040.803,44,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]",3.0,3
3,PH1011-PHYSICS_20150922__4284.434__48__B_CHART...,PH1011-PHYSICS_20150922,4284.434,48,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]",3.0,3
4,PH1011-PHYSICS_20150922__4957.804__62__B_CHART...,PH1011-PHYSICS_20150922,4957.804,62,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]",3.0,3


### Add layout information

In [5]:
video_layouts = os.listdir(C.LAYOUT_ANNOTATIONS_DIR)
video_layouts = [video[:-3] for video in video_layouts]

print(len(video_layouts))
print(video_layouts[0])

179
16S2-MH1402-LEC_20170331


In [6]:
layout_tensor = torch.load(f"{C.LAYOUT_ANNOTATIONS_DIR}/PH1011-PHYSICS_20150818.pt")
layout_tensor.shape

torch.Size([13660])

In [7]:
video_tensor = torch.load(f"{ROOT}/data/Converted/PH1011-PHYSICS_20150818.pt")
video_tensor.shape

torch.Size([13660, 180, 360])

In [9]:
orig_video_clip = VideoFileClip(f"{ROOT}/data/Processed/PH1011-PHYSICS_20150818.mp4")

In [12]:
print(orig_video_clip.duration)
print(orig_video_clip.fps)


7198.23
50.0


In [15]:
events_df = pd.read_csv(C.PROCESSED_ANNOTATIONS_PATH, converters=C.A_CONVERTERS)
video_names = events_df[C.A_VIDEO].unique()

print(len(video_names))
print(video_names[0])

118
PH1011-PHYSICS_20150922


In [47]:
video_durations = {}

for video in video_names:
    
    video_path = f"{C.ORIGINAL_VIDEOS_DIR}/{video}.mp4"
    video_clip = VideoFileClip(video_path)
    
    video_durations[video] = video_clip.duration

In [98]:
def get_frame_layout(frame_row):
    video = frame_row[C.F_VIDEO]
    
    if video not in video_layouts:
        print(f"No layout for video: {video}")
        return None

    frame_time = frame_row[C.F_TIME]
    video_length = video_durations[video]
    
    layouts_torch = torch.load(f"{C.LAYOUT_ANNOTATIONS_DIR}/{video}.pt")
    number_of_video_layout_annotations = layouts_torch.shape[0]
#     print(f"{frame_time} {video_length} {number_of_video_layout_annotations}")
              
    layout_index_approximation = (frame_time / video_length) * number_of_video_layout_annotations
    layout_index_approximation = int(layout_index_approximation)
    layout = layouts_torch[layout_index_approximation].item()
    layout = int(layout)
    
    return layout

In [99]:
frames_df[C.F_LAYOUT] = frames_df.apply(lambda row: get_frame_layout(row), axis=1)

### Filter out frames without layouts

In [115]:
len(frames_df)

4920

In [116]:
frames_with_layout = frames_df[frames_df[C.F_LAYOUT].notnull()]
len(frames_with_layout)

4920

In [117]:
frames_with_layout.to_csv(C.Frames.FRAMES_DF_PATH, index=False)

### Add layouts and splits

In [118]:
frames_df = pd.read_csv(C.Frames.FRAMES_DF_PATH, converters=C.F_CONVERTERS)
frames_df.head(1)

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layouts,Layout
0,PH1011-PHYSICS_20150922__3435.269__40__B_CHART...,PH1011-PHYSICS_20150922,3435.269,40,B_CHARTS_P,RA13,"[39, 126]","[B_CHARTS_S, B_CHARTS_P]",3.0,3


In [119]:
def get_split_layouts(layout):
    # layout < 3 => whole screen is either Camera or Screen
    if layout < 3:
        layouts = [layout, layout]
        
    # layout < 7 => layout is divided into 3 parts:
    # - 1 vertical line in 1/3 of width middle that splits screen to L and R
    # - 1 horizontal line in the middle of L
    elif layout < 7:
        layouts = [[1,1], [1,2], [2,1], [2,2]][layout - 3]
    # layout >=7 => layout is split in half
    else:
        layouts = [[1,1], [1,2], [2,1], [2,2]][layout - 7]
    
    return layouts

In [120]:
frames_df[C.F_LAYOUTS] = frames_df.apply(lambda row: get_split_layouts(row[C.F_LAYOUT]), axis=1)

In [121]:
def get_splits(frame_row):
    frame_id = frame_row[C.F_ID]
    layout = frame_row[C.F_LAYOUT]
    
    if layout < 3:
        return [f"{frame_id}__Full", f"{frame_id}__Full"]
    
    return [f"{frame_id}__Left", f"{frame_id}__Right"]

In [122]:
frames_df[C.F_SPLITS] = frames_df.apply(lambda row: get_splits(row), axis=1)

In [123]:
frames_df.head(1)

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layouts,Layout,Splits
0,PH1011-PHYSICS_20150922__3435.269__40__B_CHART...,PH1011-PHYSICS_20150922,3435.269,40,B_CHARTS_P,RA13,"[39, 126]","[B_CHARTS_S, B_CHARTS_P]","[1, 1]",3,[PH1011-PHYSICS_20150922__3435.269__40__B_CHAR...


In [124]:
frames_df.to_csv(C.Frames.FRAMES_DF_PATH, index=False)